1. Import libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from mpl_toolkits.mplot3d import Axes3D

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn import tree

import warnings
warnings.filterwarnings("ignore")
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


2. Import data

In [ ]:
directory = "/kaggle/input/airline-passenger-satisfaction/"
feature_tables = ['train.csv', 'test.csv']

df_train = directory + feature_tables[0]
df_test = directory + feature_tables[1]

# Create dataframes
print(f'Reading csv from {df_train}...')
train = pd.read_csv(df_train)
print('...Complete')

print(f'Reading csv from {df_train}...')
test = pd.read_csv(df_test)
print('...Complete')

df = pd.concat([train, test])

Notes about the Dataset

This dataset has already been split into train and test csv files. 80% of the total dataset is in train.csv and 20% is in test.csv

3. Information about dataset

Depict the dataset

In [ ]:
train.head()

Get summary of data

In [ ]:
print(train.info())

Data type of each column

In [ ]:
print(train.dtypes)

In [ ]:
print(train.describe(include = 'all'))

Get description of data - statistical summary

Shape of the dataframe

In [ ]:
print(train.shape)

Get number of NaN values

In [ ]:
print(train.isnull().sum())

Clean and Transform Dataset



In [ ]:
def transform_gender(x):
    if x == 'Female':
        return 1
    elif x == 'Male':
        return 0
    else:
        return -1
    
def transform_customer_type(x):
    if x == 'Loyal Customer':
        return 1
    elif x == 'disloyal Customer':
        return 0
    else:
        return -1
    
def transform_travel_type(x):
    if x == 'Business travel':
        return 1
    elif x == 'Personal Travel':
        return 0
    else:
        return -1
    
def transform_class(x):
    if x == 'Business':
        return 2
    elif x == 'Eco Plus':
        return 1
    elif x == 'Eco':
        return 0    
    else:
        return -1
    
def transform_satisfaction(x):
    if x == 'satisfied':
        return 1
    elif x == 'neutral or dissatisfied':
        return 0
    else:
        return -1
    
def process_data(df):
    df = df.drop(['Unnamed: 0', 'id'], axis = 1)
    df['Gender'] = df['Gender'].apply(transform_gender)
    df['Customer Type'] = df['Customer Type'].apply(transform_customer_type)
    df['Type of Travel'] = df['Type of Travel'].apply(transform_travel_type)
    df['Class'] = df['Class'].apply(transform_class)
    df['satisfaction'] = df['satisfaction'].apply(transform_satisfaction)
    df['Arrival Delay in Minutes'].fillna(df['Arrival Delay in Minutes'].median(), inplace = True)
    
    return df

train = process_data(train)
test = process_data(test)

In [ ]:
train.head()

In [ ]:
print(train.isnull().sum())

In [ ]:
train.describe()

In [ ]:
labelencoder = LabelEncoder()
train['satisfaction'] = labelencoder.fit_transform(train['satisfaction'])
test['satisfaction'] = labelencoder.fit_transform(test['satisfaction'])

Does satisfaction depend on gender?

Gender
1 - Female
0 - Male

In [ ]:
print('How many people satisfacted by Airline:')
train.groupby('Gender')[['satisfaction']].sum()

In [ ]:
print('How many people participated in the study:')
train.groupby('Gender')[['satisfaction']].count()

In [ ]:
print('Percentage of satisfacted people :')
train.groupby('Gender')[['satisfaction']].sum()/ train.groupby('Gender')[['satisfaction']].count()

So, people`s satisfaction does not depend on gender.

**Age**

In [ ]:
pd.crosstab(train['satisfaction'], train['Age'], dropna=True, normalize='columns')


In [ ]:
k = pd.crosstab(train['Age'],train['satisfaction'], dropna=True, normalize='columns')
k.plot.bar(stacked=False, figsize=(25, 5))
print('Age-satisfaction')
plt.show()

Does satisfaction depend on the Class?

Class
2 - Bisiness
1 - Eco
0 - EcoPlus

In [ ]:
print ('Satisfaction of people depending on the class:')
train.groupby('Class')[['satisfaction']].sum()

In [ ]:
print ('How many people participated in the study:')
train.groupby('Class')[['satisfaction']].count()

In [ ]:
print ('Percentage of satisfacted people depending from the class:')
train.groupby('Class')[['satisfaction']].sum()/ train.groupby('Class')[['satisfaction']].count()

So, satisfaction depends on class: Business class passengers have the highest percantage of satisfaction.

*Correlation*

In [ ]:
corr = train.corr(method='spearman')

mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True


f, ax = plt.subplots(figsize=(25, 25))


sns.heatmap(corr, annot = True, mask=mask, cmap="YlGnBu", center=0,
            square=True, linewidths=.5)

In [ ]:
for a in corr.columns:
    for b in corr.index:
        if (a != b) and (abs(corr[a][b]) >= 0.7):
            print(a,b,'-->',corr[a][b])

*Logistic Regression*

In [ ]:
features = ['Gender', 'Customer Type', 'Age', 'Type of Travel', 'Class',
       'Flight Distance', 'Inflight wifi service',
       'Departure/Arrival time convenient', 'Ease of Online booking',
       'Gate location', 'Food and drink', 'Online boarding', 'Seat comfort',
       'Inflight entertainment', 'On-board service', 'Leg room service',
       'Baggage handling', 'Checkin service', 'Inflight service',
       'Cleanliness', 'Departure Delay in Minutes', 'Arrival Delay in Minutes']
target = 'satisfaction'
X =  train [features]
y = train [target]

X_train = train[features]
y_train = train[target].to_numpy()
X_test = test[features]
y_test = test[target].to_numpy()

In [ ]:
# data normalization with sklearn
from sklearn.preprocessing import MinMaxScaler
# fit scaler on training data
norm = MinMaxScaler().fit(X_train)
# transform training data
X_train_norm = norm.transform(X_train)
# transform testing dataabs
X_test_norm = norm.transform(X_test)

In [ ]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(random_state=0)
logreg.fit(X_train,y_train)
y_pred=logreg.predict(X_test)
print(y_pred)

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, y_pred)
print(confusion_matrix)

The result is telling us that we have 11288+9215 correct predictions and 2188+3285 incorrect predictions.

In [ ]:
# Create a matrix 2x2 with a code:
class_names=[0,1] # name  of classes
fig, ax = plt.subplots()
tick_marks = np.arange(len(class_names))
plt.xticks(tick_marks, class_names)
plt.yticks(tick_marks, class_names)
sns.heatmap(pd.DataFrame(confusion_matrix), annot=True, cmap="YlGnBu" ,fmt='g')
ax.xaxis.set_label_position("top")
plt.tight_layout()
plt.title('Confusion matrix', y=1.1)
plt.ylabel('Actual label')
plt.xlabel('Predicted label')

In [ ]:
# evaluate the model using model evaluation metrics 
# such as accuracy, precision, and recall
from sklearn import metrics
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print("Precision:",metrics.precision_score(y_test, y_pred))
print("Recall:",metrics.recall_score(y_test, y_pred))

In [ ]:
# ROC Curve:
# Receiver Operating Characteristic(ROC) curve is a plot of the true 
# positive rate against the false positive rate. It shows the tradeoff 
# between sensitivity and specificity.
y_pred_proba = logreg.predict_proba(X_test)[::,1]
fpr, tpr, _ = metrics.roc_curve(y_test,  y_pred_proba)
auc = metrics.roc_auc_score(y_test, y_pred_proba)
plt.plot(fpr,tpr,label="data 1, auc="+str(auc))
plt.legend(loc=4)
plt.show()

In [ ]:
# ROC Curve: 0.849
# AUC score for the case is 0.849. AUC score 1 represents perfect classifier, 
# and 0.5 represents a worthless classifier.

In [ ]:
y_pred = logreg.predict(X_test)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
logit_roc_auc = roc_auc_score(y_test, logreg.predict(X_test))
fpr, tpr, thresholds = roc_curve(y_test, logreg.predict_proba(X_test)[:,1])
plt.figure()
plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % logit_roc_auc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.savefig('Log_ROC')
plt.show()

In [ ]:
# intercept of the model
print('Intercept:', logreg.intercept_)

In [ ]:
import lightgbm as lgb
clf = lgb.LGBMClassifier()
clf.fit(X_train , y_train)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(clf.predict(X_train) , y_train)

In [ ]:
accuracy_score(clf.predict(X_test) , y_test)

In [ ]:
import shap

In [ ]:
explainer = shap.TreeExplainer(clf)
shap_values = explainer.shap_values(X_train)
shap.summary_plot(shap_values, X_train)

Choosing 10 top features

In [ ]:
from sklearn import preprocessing
r_scaler = preprocessing.MinMaxScaler()
r_scaler.fit(train)
#modified_data = pd.DataFrame(r_scaler.transform(train), index=train['id'], columns=train.columns)
modified_data = pd.DataFrame(r_scaler.transform(train), columns=train.columns)
modified_data.head()

In [ ]:
from sklearn.feature_selection import SelectKBest, chi2
X = modified_data.loc[:,modified_data.columns!='satisfaction']
y = modified_data[['satisfaction']]
selector = SelectKBest(chi2, k=10)
selector.fit(X, y)
X_new = selector.transform(X)
print(X.columns[selector.get_support(indices=True)])

These are top 10 features impacting on passenger satisfaction. 

Building Model

In [ ]:
features = ['Type of Travel', 'Class', 'Flight Distance', 'Inflight wifi service',
       'Online boarding', 'Seat comfort', 'Inflight entertainment',
       'On-board service', 'Leg room service', 'Cleanliness']
target = 'satisfaction'
X =  train [features]
y = train [target]


# Split into test and train
X_train = train[features]
y_train = train[target].to_numpy()
X_test = test[features]
y_test = test[target].to_numpy()

# Normalize Features
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [ ]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()


# fit the model with data
logreg.fit(X_train,y_train)
y_pred=logreg.predict(X_test)
print(y_pred)


In [ ]:
from sklearn import metrics
cnf_matrix = metrics.confusion_matrix(y_test, y_pred)
cnf_matrix

In [ ]:
import time
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report, plot_confusion_matrix, plot_roc_curve
from matplotlib import pyplot as plt 
def run_model(model, X_train, y_train, X_test, y_test, verbose=True):
    t0=time.time()
    if verbose == False:
        model.fit(X_train,y_train.ravel(), verbose=0)
    else:
        model.fit(X_train,y_train.ravel())
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred) 
    time_taken = time.time()-t0
    print("Accuracy = {}".format(accuracy))
    print("ROC Area under Curve = {}".format(roc_auc))
    print("Time taken = {}".format(time_taken))
    print(classification_report(y_test,y_pred,digits=5))
    plot_confusion_matrix(model, X_test, y_test,cmap=plt.cm.pink, normalize = 'all')
    plot_roc_curve(model, X_test, y_test)                     
    
    return model, accuracy, roc_auc, time_taken

In [ ]:
from sklearn.linear_model import LogisticRegression

params_lr = {'penalty': 'elasticnet', 'l1_ratio':0.5, 'solver': 'saga'}

model_lr = LogisticRegression(**params_lr)
model_lr, accuracy_lr, roc_auc_lr, tt_lr = run_model(model_lr, X_train, y_train, X_test, y_test)

Lasso Regression

In [ ]:
from sklearn.linear_model import Lasso
from sklearn import linear_model
features = ['Gender', 'Customer Type', 'Age', 'Type of Travel', 'Class',
       'Flight Distance', 'Inflight wifi service',
       'Departure/Arrival time convenient', 'Ease of Online booking',
       'Gate location', 'Food and drink', 'Online boarding', 'Seat comfort',
       'Inflight entertainment', 'On-board service', 'Leg room service',
       'Baggage handling', 'Checkin service', 'Inflight service',
       'Cleanliness', 'Departure Delay in Minutes', 'Arrival Delay in Minutes']
target = 'satisfaction'
X =  train [features]
y = train [target]



# Split into test and train
X_train = train[features]
y_train = train[target]
X_test = test[features]
y_test = test[target]
X_train,X_test,y_train,y_test=train_test_split(X,y, test_size=0.2, random_state=42)
lasso = Lasso()
lasso.fit(X_train,y_train)
train_score=lasso.score(X_train,y_train)
test_score=lasso.score(X_test,y_test)
coeff_used = np.sum(lasso.coef_!=0)

print ("Training score:", train_score) 
print ("Test score:", test_score)
print ("Number of features used:", coeff_used)

In [ ]:
lasso001 = Lasso(alpha=0.01, max_iter=10e5)
lasso001.fit(X_train,y_train)
train_score001=lasso001.score(X_train,y_train)
test_score001=lasso001.score(X_test,y_test)
coeff_used001 = np.sum(lasso001.coef_!=0)

print ("Training score for alpha = 0.01:", train_score001)
print ("Test score for alpha = 0.01:", test_score001)
print ("Number of features used: for alpha = 0.01:", coeff_used001)

In [ ]:
lasso00001 = Lasso(alpha=0.0001, max_iter=10e5)
lasso00001.fit(X_train,y_train)
train_score00001=lasso00001.score(X_train,y_train)
test_score00001=lasso00001.score(X_test,y_test)
coeff_used00001 = np.sum(lasso00001.coef_!=0)

print ("Training score for alpha = 0.0001:", train_score00001)
print ("Test score for alpha = 0.0001:", test_score00001)
print ("Number of features used: for alpha = 0.0001:", coeff_used00001)

In [ ]:
logreg = LogisticRegression()
logreg.fit(X_train,y_train)
logreg_train_score=logreg.score(X_train,y_train)
logreg_test_score=logreg.score(X_test,y_test)
print ("Logistic Regression training score:", logreg_train_score)
print ("Logistic Regression  test score:", logreg_test_score)


In [ ]:
plt.subplot(1,2,1)
plt.plot(lasso.coef_,alpha=0.7,linestyle='none',marker='*',markersize=6,color='red',label=r'Lasso; $\alpha = 1$',zorder=7) # alpha here is for transparency
plt.plot(lasso001.coef_,alpha=0.5,linestyle='none',marker='d',markersize=7,color='blue',label=r'Lasso; $\alpha = 0.01$')

plt.xlabel('Coefficient Index',fontsize=16)
plt.ylabel('Coefficient Magnitude',fontsize=16)
plt.legend(fontsize=13,loc=4)
plt.subplot(1,2,2)
plt.plot(lasso.coef_,alpha=0.7,linestyle='none',marker='*',markersize=6,color='red',label=r'Lasso; $\alpha = 1$',zorder=7)
plt.plot(lasso001.coef_,alpha=0.5,linestyle='none',marker='d',markersize=7,color='blue',label=r'Lasso; $\alpha = 0.01$')
plt.plot(lasso00001.coef_,alpha=0.8,linestyle='none',marker='v',markersize=7,color='black',label=r'Lasso; $\alpha = 0.00001$')
plt.plot(logreg.coef_,alpha=0.7,linestyle='none',marker='o',markersize=6,color='green',label='Linear Regression',zorder=2)
plt.xlabel('Coefficient Index',fontsize=16)
plt.ylabel('Coefficient Magnitude',fontsize=16)
plt.legend(fontsize=13,loc=4)
plt.tight_layout()
plt.show()

In [ ]:
from sklearn.linear_model import Lasso
from sklearn import linear_model
features = ['Gender', 'Customer Type', 'Age', 'Type of Travel', 'Class',
       'Flight Distance', 'Inflight wifi service',
       'Departure/Arrival time convenient', 'Ease of Online booking',
       'Gate location', 'Food and drink', 'Online boarding', 'Seat comfort',
       'Inflight entertainment', 'On-board service', 'Leg room service',
       'Baggage handling', 'Checkin service', 'Inflight service',
       'Cleanliness', 'Departure Delay in Minutes', 'Arrival Delay in Minutes']
target = 'satisfaction'
X =  train [features]
y = train [target]

# Split into test and train
X_train = train[features]
y_train = train[target]
X_test = test[features]
y_test = test[target]


X_train,X_test,y_train,y_test=train_test_split(X,y, test_size=0.2, random_state=42)
lassomodel = Lasso(alpha=0.01)
lassomodel.fit(X_train,y_train)

print('Intercept:', lassomodel.intercept_)
print("R-squared:", lassomodel.score(X_train,y_train))

#predictions = lassomodel.predict(X)
#print(predictions)

dataframe = pd.DataFrame({"Feature":X.columns.tolist(),"Coefficients":lassomodel.coef_})
dataframe.sort_values("Coefficients", ascending = False)

*Desicion Tree*

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
from sklearn.tree import DecisionTreeRegressor
DecTree=DecisionTreeRegressor(min_samples_leaf=.0001)
DecTree.fit(X_train,y_train)

print('R-squared score - training: {:.3f}'.format(DecTree.score(X_train,y_train)))
print('R-squared score - test: {:.3f}'.format(DecTree.score(X_test, y_test)))

In [ ]:
decision_tree = tree.DecisionTreeClassifier(criterion='entropy', max_depth = 10, random_state=15)
decision_tree.fit(X_train, y_train)

In [ ]:
from IPython.display import Image as PImage
from subprocess import check_call
from PIL import Image, ImageDraw
import graphviz  
from sklearn.tree import export_graphviz

# Export our trained model as a .dot file
with open("tree1.dot", 'w') as f:
     f = export_graphviz(decision_tree, out_file=f, max_depth = 5,
                         impurity = True, feature_names = X_train.columns,
                         rounded = True, filled= True )
#Convert .dot to .png to allow display in web notebook
check_call(['dot','-Tpng','tree1.dot','-o','tree.png'])
# Annotating chart with PIL
img = Image.open("tree.png")
draw = ImageDraw.Draw(img)
img.save('sample-out.png')
PImage("sample-out.png")

In [ ]:
decision_tree.score(X_test, y_test)